In [1]:
import pandas as pd
import pickle


### Code to build extrinsic model based on user data
* User data is difficult to pull - only used 1700 users in model (proof of concept?)

### Read in user URLs (copied manually) and clean up - can be any number of digits

In [2]:
##read in user URLS from csv and clean up
df = pd.read_csv("users.csv",names=['user_url'])

In [3]:
def convert(s):
    import re
    return re.findall('\d+', s)
    

In [4]:
df['userId'] = df.apply(lambda x:convert(x['user_url'][37:46]), axis=1)

In [5]:
df = df.groupby(['user_url','userId']).count()

In [6]:
df.reset_index(inplace=True)

### Function to get user's last 100 ticks from mountainproject API
* Get your own API key at mountainproject.com

In [7]:
def get_ticks(email = None, userId = None):
    import requests
    import json
    if email is None and userId is None:
        print('Error - nothing passed')
        return []

    key = 'your_api_key_goes_here'
    if email is not None:
        if userId is not None:
            print('Input UserID OR Email')
            return []
        else:
            url = 'https://www.mountainproject.com/data/get-ticks?email=' + email + '&key=' + key   
    else:
        url = 'https://www.mountainproject.com/data/get-ticks?userId=' + str(userId) + '&key=' + key
        
    r = requests.get(url)

    data = r.json()

    start_list = []

    for route in data['ticks']:
        start_list.append(route['routeId'])
    
    return start_list

### Iterate through all users to get ticks, put it in a dictionary
* Pickle it for later so you don't have to do this again
* Be nice to mountainproject, they are a great resource!!!

In [ ]:
#get user ticks, put them in a dictionary
d = {}
for user in df.userId:
    d[user] = get_ticks(userId = user)
    

In [ ]:
with open('user_ticks_dict.pkl', 'wb') as picklefile:
    pickle.dump(d, picklefile)

In [9]:
with open("user_ticks_dict.pkl", 'rb') as picklefile: 
    d = pickle.load(picklefile)

### Cleaning up users/routes and building sparse matrix

In [10]:
#get unique routes from all users
route_list = []
for key in d:
    route_list.extend(d[key])

unique_routes = list(set(route_list))

In [11]:
#create list of users
user_list = []
for key in d:
    user_list.append(key)

In [12]:
#create sparse dictionary of all users & routes
from collections import defaultdict
d2 = defaultdict(list)
for user in user_list:
    for route in unique_routes:
        if route in d[user]:
            d2[user].append(1)
        else:
            d2[user].append(0)

In [13]:
#make it a dataframe!
df2 = pd.DataFrame.from_dict(d2)

### NMF to get probabilities and components - output dense matrix
* Use dot product of these output matricies to build dense matrix from sparse matrix
* This is one of my favorite things - see presentation for explanation!

In [14]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=20)
probs = nmf.fit_transform(df2)

In [15]:
filled_array = np.dot(probs,nmf.components_)

In [16]:
df_out = pd.DataFrame(filled_array,columns=user_list, index=unique_routes)

In [19]:
with open('user_prefs.pkl', 'wb') as picklefile:
    pickle.dump(df_out, picklefile)